In [ ]:
%matplotlib nbagg
import numpy as np
import json
import os
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

from alvra_tools import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm
from alvra_tools.channels import *

In [ ]:
scan_name ="MB_jetscan_19"
scan_range =".run002940-002950"


DIR = "/sf/alvra/data/p18442/raw/scan_data/" + scan_name + "/"
DIR_json = "/sf/alvra/data/p18442/res/scan_info/"

In [ ]:
!ls -lah -rt $DIR
!hostname

listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files,"files in the folder", DIR)

In [ ]:
#json_file = DIR_json + scan_name + "_scan_info.json"
json_file = DIR_json + scan_name + scan_range + ".json"

print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 17

print ("Processing",numFiles,"files")

Signal_trans = []
err_Signal_trans = []

Signal_fluo = []
err_Signal_fluo = []

Position_mm = []


for i in range(0,numFiles):
    filename = str(data['scan_files'][i][0])
    exists = os.path.isfile(filename)
    if exists: #(and i!=3):
        print("step",i+1,"of",numFiles,": Processing %s" % (str(data['scan_files'][i][0])))
        
        (data_t_shot, data_f_shot, Izero_shot, laser_shot, position_shot, pulseIDs) = \
        load_FEL_scans(filename, channel_position_X)
        
        data_t_shot_norm = data_t_shot / Izero_shot
        data_f_shot_norm = data_f_shot / Izero_shot
        
        Signal_trans.append(data_t_shot_norm.mean())
        err_Signal_trans.append(data_t_shot_norm.std()/np.sqrt(data_t_shot_norm.size))
        
        Signal_fluo.append(data_f_shot_norm.mean())
        err_Signal_fluo.append(data_f_shot_norm.std()/np.sqrt(data_f_shot_norm.size))
    
        position_shot = [x for x in position_shot if (np.abs(x) > 0)]
        Position_mm.append(np.mean(position_shot))
        #Position_mm.append(position_loop[0])
        
        #print (Position_mm, Signal_fluo)
    
Signal_trans = np.asarray(Signal_trans)
err_Signal_trans = np.asarray(err_Signal_trans)

Signal_fluo = np.asarray(Signal_fluo)
err_Signal_fluo = np.asarray(err_Signal_fluo)

Position_mm = np.asarray(Position_mm)
print ("Job done!")

In [ ]:
plt.figure(figsize = (6,10))
plt.subplots_adjust(hspace=0.25)

plt.subplot(211)
plt.plot(Position_mm, Signal_trans, label="Transmission",marker='.')
low_err_t= Signal_trans - err_Signal_trans
high_err_t= Signal_trans + err_Signal_trans
plt.fill_between(Position_mm, low_err_t, high_err_t, color='lightblue')
plt.title(DIR)
plt.grid()
plt.xlabel("Position (mm)")
plt.legend(loc="best")

plt.subplot(212)
plt.plot(Position_mm, Signal_fluo, label="Fluorescence",marker='.')
low_err_f= Signal_fluo - err_Signal_fluo
high_err_f= Signal_fluo + err_Signal_fluo
plt.fill_between(Position_mm, low_err_f, high_err_f, color='lightblue')
plt.title(DIR)
plt.grid()
plt.xlabel("Position (mm)")
plt.legend(loc="best")

plt.show()
#plt.savefig('Timescan_7_diode')